In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
from scipy.stats import gaussian_kde



In [2]:
df_t = pd.read_csv('data/cleaned/cleaned_trips.csv', parse_dates=['datetime'], infer_datetime_format=True)

In [103]:
len(df_t.ix[(df_t['tripduration'] >= 4) & (df_t['tripduration'] <= 60),:])
len(df_t.ix[(df_t['from_station_id'] != df_t['to_station_id']) & (df_t['tripduration'] >= 4) & (df_t['tripduration'] <= 60), :])


116268

In [3]:
df_t['hour'].unique()

array([10, 11, 12, 13, 14, 15, 17, 19,  7,  8,  9, 16, 22, 23, 21,  0,  1,
       20, 18,  4,  3,  2,  6,  5])

In [27]:
plt.hist(df_t['hour'], bins=50)
plt.show()

In [ ]:
pd.cut(df_t['hour'],[0,6,10,15,19] , include_lowest=True, right=True)

In [28]:
#df_t['hour'] = pd.cut(df_t['hour'],[0,6,10,15,19] , include_lowest=True, right=True)

NameError: name 'df_w' is not defined

In [15]:
eco_dict = {'Seattle': 0, 'Udist': 1, 'Outliers': -1}
eco_list=['Seattle']
eco_list = [eco_dict[x] for x in eco_list]

In [17]:
len(df_t.ix[(df_t['ecosystem_to'].isin(eco_list)) & (df_t['ecosystem_from'].isin(eco_list)),:])

124339

In [26]:
usertypes=['Short-Term Pass Holder']
df_t.ix[df_t['usertype'].isin(usertypes),:]

55230

In [35]:
print list(df_w.columns)

['datetime', 'TEMP', 'SPD', 'PCP01', 'CLG', 'DIR', 'VSB', 'DEWP', 'SLP', 'STP']


In [38]:
df_w.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9529 entries, 0 to 9528
Data columns (total 10 columns):
datetime    9529 non-null datetime64[ns]
TEMP        9529 non-null float64
SPD         9528 non-null float64
PCP01       9073 non-null float64
CLG         9527 non-null float64
DIR         8327 non-null float64
VSB         9529 non-null float64
DEWP        9529 non-null float64
SLP         9528 non-null float64
STP         9528 non-null float64
dtypes: datetime64[ns](1), float64(9)
memory usage: 818.9 KB


In [ ]:
df_t.ix[df_t['ecosystem_to'].isin()]

In [ ]:
def remove_trip_outliers(df, duration=[3, 60], same=False):
    # subset by min/max duration
    df = df.ix[(df_t['tripduration'] >= duration[0]) & (df_t['tripduration'] <= duration[1]),:]
    
    #remove trips with same origin and destination
    if same:
        df = df.ix[df_t['from_station_id'] != df_t['to_station_id'], :]
    return df

In [97]:
len(df_t)

141943

In [30]:
df_w = pd.read_csv('data/cleaned/cleaned_weather.csv', parse_dates=['datetime'], infer_datetime_format=True)

In [70]:
df_c = pd.read_csv('data/raw/clusters.csv')

In [69]:
df_c.rename(columns={'big_cluster': 'ecosystem', 'smallgroups': 'cluster'}, inplace = True)
df_c.to_csv('data/raw/clusters.csv', index= False)

In [104]:
# df = df_t.merge(df_c[['dockcount','elevation','cluster','ecosystem','terminal']], left_on = 'from_station_id', right_on = 'terminal',  how='left')
# # df.drop('terminal', axis=1, inplace=True)
# df.merge(df_c[['terminal', 'cluster', 'ecosystem']], left_on ='to_station_id', right_on='terminal', suffixes=['_from','_to'])

In [131]:
# Convert columns and create hour column
#df_trips['date'] = df_trips['starttime'].str.replace('\s.*','')
df_trips['starttime'] = pd.to_datetime(df_trips['starttime'])

In [135]:
df_trips['tripduration'] = df_trips['tripduration'] / float(60)

In [1]:
df_trips = df_trips.drop(['from_station_name', 'to_station_name', 'trip_id'], axis = 1)


df_trips['hour'] = df_trips['starttime'].map(lambda x: x.hour)

df_trips = df_trips.ix[(df_trips['tripduration'] > 4) & (df_trips['tripduration'] < 120) , :]
df_trips = df_trips.ix[(df_trips['usertype'] =='Annual Member') &  (df_trips['from_station_id'] != df_trips['to_station_id']), :]

# Group by to get totals per day, per hour, per station
#df= df_trips.groupby(['date','hour','from_station_id'])['bikeid'].count().reset_index()

df = df_trips
# rename columns 
df = df.rename(columns = {'from_station_id': 'terminal', 'bikeid': 'count'})

#remove pronto shop columns
df = df.ix[~df['terminal'].isin(['UW-01','Pronto shop']),:]
df['count'] = 1

# Get list of terminals and hours
terminals = df['terminal'].unique()
hours = df['hour'].unique()

Index([u'trip_id', u'starttime', u'stoptime', u'bikeid', u'tripduration',
       u'from_station_name', u'to_station_name', u'from_station_id',
       u'to_station_id', u'usertype', u'gender', u'birthyear'],
      dtype='object')


In [3]:
df_h = pd.read_table('../open_data_year_one/hw.txt',sep=' ', error_bad_lines= False, index_col=False)


In [97]:
df_h.ix[]

,USAF,WBAN,YR--MODAHRMN,DIR,SPD,GUS,CLG,SKC,L,M,...,SLP,ALT,STP,MAX,MIN,PCP01,PCP06,PCP24,PCPXX,SD
0,727930,24233,201410010000,020,8,***,***,BKN,*,*,...,1019.8,*****,1002.9,67,54,*****,*****,*****,*****,**
1,727930,24233,201410010053,040,8,***,722,SCT,*,*,...,1019.9,30.10,1003.5,***,***,0.00,*****,*****,*****,**
2,727930,24233,201410010153,080,5,***,722,SCT,*,*,...,1020.4,30.12,1004.1,***,***,0.00,*****,*****,*****,**
3,727930,24233,201410010253,070,6,***,75,***,*,*,...,1021.0,30.14,1004.8,***,***,0.00,*****,*****,*****,**
4,727930,24233,201410010353,080,8,***,80,***,*,*,...,1021.6,30.15,1005.1,***,***,0.00,*****,*****,*****,**


In [4]:
df_h = df_h.replace('\*+|T', np.nan, regex=True)

In [40]:
df_h.ix[df_h['TEMP'] == '****', 'TEMP'] = np.nan
df_h.ix[df_h['SPD'] == '***', 'SPD'] = np.nan
df_h.ix[df_h['PCP01'].isin(['****','0.00T*****','0.00T','*****']), 'PCP01'] = np.nan
df_h.ix[df_h['CLG'] == '***', 'CLG'] = np.nan

In [41]:
df_h['TEMP'] = df_h['TEMP'].astype(float)
df_h['TEMP'].hist()
plt.show()

In [5]:
df_h['SPD'] = df_h['SPD'].astype(float)
df_h['SPD'].hist()
#plt.show()

In [45]:
df_h['PCP01'] =  df_h['PCP01'].astype(float)
df_h['PCP01'].hist(bins = 100)
#plt.show()

In [46]:
df_h['CLG'] =  df_h['CLG'].astype(float)
df_h['CLG'].hist(bins = 100)
#plt.show()

In [ ]:
df_h = df_h[['TEMP','SPD','PCP01','CLG','DIR','VSB','DEWP','SLP','STP']].astype(float)

In [239]:
df_h['YR--MODAHRMN'][:5]

0    201410010000
1    201410010053
2    201410010153
3    201410010253
4    201410010353
Name: YR--MODAHRMN, dtype: int64

In [262]:
x = pd.to_datetime(df_h['YR--MODAHRMN'][:10], format='%Y%m%d%H%M')

In [290]:
def round_to_hour(x):
    if x.minute > 30:
        x = x.replace(minute=0)
        x = x + pd.offsets.Hour(1)
        
    else:
        x = x.replace(minute=0)
    return x

In [291]:
map(round_to_hour,x)

[Timestamp('2014-10-01 00:00:00'),
 Timestamp('2014-10-01 01:00:00'),
 Timestamp('2014-10-01 02:00:00'),
 Timestamp('2014-10-01 03:00:00'),
 Timestamp('2014-10-01 04:00:00'),
 Timestamp('2014-10-01 05:00:00'),
 Timestamp('2014-10-01 06:00:00'),
 Timestamp('2014-10-01 06:00:00'),
 Timestamp('2014-10-01 07:00:00'),
 Timestamp('2014-10-01 08:00:00')]

In [248]:
round_to_hour(df_h['YR--MODAHRMN'][:10])

DatetimeIndex([   '1970-01-01 00:03:21.410010',
               '1970-01-01 00:03:21.410010053',
               '1970-01-01 00:03:21.410010153',
               '1970-01-01 00:03:21.410010253',
               '1970-01-01 00:03:21.410010353',
               '1970-01-01 00:03:21.410010453',
               '1970-01-01 00:03:21.410010553',
               '1970-01-01 00:03:21.410010600',
               '1970-01-01 00:03:21.410010653',
               '1970-01-01 00:03:21.410010751'],
              dtype='datetime64[ns]', freq=None, tz=None)

In [102]:
df_h['rounded_time'] =pd.to_datetime(df_h['rounded_time'], format='%Y%m%d%H%M')

In [48]:
df_h = df_h[['rounded_time','TEMP','CLG','PCP01','SPD']]

In [49]:
df_h = df_h.groupby(['rounded_time']).mean().reset_index()

,rounded_time,TEMP,CLG,PCP01
0,201410010000,65,722,0
1,201410010100,62,722,0
2,201410010200,61,75,0
3,201410010300,61,80,0
4,201410010400,59,55,0


In [12]:
datetimes =  df_h['rounded_time']

In [13]:
# create new full index and reindex
full_index = pd.MultiIndex.from_product([datetimes, terminals], names=['datetime', 'terminal'])

In [14]:
df_h['terminal'] = 'BT-01'
df_h.index = [df_h.rounded_time, df_h.terminal]

In [15]:
df_h.drop(['terminal','rounded_time'], axis = 1, inplace = True)
df_h = df_h.reindex(full_index)
df_h = df_h.reset_index()
df_h = df_h.ix[:438000,:]

In [16]:
df_h['datetime'] = df_h['datetime'].astype(str)

In [17]:
count = 0
s = 1
for date in df_h['datetime'].unique()[8001:]:
    df_h.ix[df_h.datetime == date, ~df_h.columns.isin(['datetime','terminal'])] = df_h.ix[(df_h.datetime == date) & (df_h.terminal == 'BT-01'), ~df_h.columns.isin(['datetime','terminal'])].values
    count += 1
    if count > 1000 * s:
        s += 1
        print count

In [18]:
df['date'] = pd.to_datetime(df['date'])

In [19]:
df['datetime'] = df['date'] + map(lambda x:pd.to_timedelta(int(x), unit='h'), df.hour)

In [20]:
df_h['datetime'] = pd.to_datetime(df_h['datetime'])

In [21]:
df = df[['terminal', 'datetime', 'count']]

In [22]:
df_final = df_h.merge(df, on = ['datetime','terminal'], how = 'left')

In [23]:
df_final['count'] = df_final['count'].fillna(0)

In [24]:
df_final.to_csv('annnual_members.csv', index= False)

In [37]:
#dff = pd.read_csv('weather_count.csv')

In [1]:
df_c = pd.read_csv('../clusters.csv')
df_t.merge(df_c[['dockcount','elevation','smallgroups','big_cluster','terminal']], on = 'terminal', how='left')

NameError: name 'pd' is not defined

In [27]:
#dff = pd.read_csv('weather_count.csv')

In [28]:
dff['datetime'] = pd.to_datetime(dff['datetime'])
dff['hour'] = map(lambda x: x.hour, dff['datetime'])
dff['dow'] = map(lambda x: x.dayofweek, dff['datetime'])
dff['month'] = map(lambda x: x.month, dff['datetime'])

In [216]:
hour_average = dff.groupby(['hour','terminal'])['count'].mean().reset_index()
hour_average.columns = ['hour','terminal', 'hour_avg']

month_average = dff.groupby(['month','terminal'])['count'].mean().reset_index()
month_average.columns = ['month','terminal', 'month_avg']

dow_average = dff.groupby(['dow','terminal'])['count'].mean().reset_index()
dow_average.columns = ['dow','terminal', 'dow_avg']

sg_hour = dff.groupby(['hour','smallgroups'])['count'].mean().reset_index()
sg_hour.columns = ['hour','smallgroups', 'sg_avg']

In [294]:
dff = dff.sort('datetime')
df_a = dff.merge(hour_average, on = ['hour','terminal']).merge(month_average, on = ['month', 'terminal']).merge(dow_average, on =['dow','terminal']).merge(sg_hour, on =['hour','smallgroups'])

In [219]:
df_a.to_csv('BEST_DATA.csv', index = False)

In [226]:
df_a = df_a.drop('count', axis = 1)

df_test = df_a.merge(df, on = ['datetime','terminal'], how = 'left')

In [234]:
df_test.to_csv('df_test.csv', index = False)

In [30]:
dff.head()

,datetime,terminal,TEMP,CLG,PCP01,SPD,count,dockcount,elevation,smallgroups,big_cluster,hood,hour,dow,month
0,2014-10-01,CBD-06,NaN,NaN,NaN,NaN,0,20,25.305275,4,0,CBD,0,2,10
1,2014-10-01,PS-04,NaN,NaN,NaN,NaN,0,18,8.136753,4,0,PS,0,2,10
2,2014-10-01,CBD-07,NaN,NaN,NaN,NaN,0,18,31.532837,4,0,CBD,0,2,10
3,2014-10-01,PS-05,NaN,NaN,NaN,NaN,0,18,11.623115,4,0,PS,0,2,10
4,2014-10-01,CBD-05,NaN,NaN,NaN,NaN,0,20,9.812873,4,0,CBD,0,2,10


In [177]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

In [295]:
df_a = df_a.ix[(df_a['hood'] != 'UW') & (df_a['hour'].isin([7,8,9,10,11,4,5,6,7])), :]
df_r = df_a[['TEMP', 'CLG', 'PCP01', 'SPD', 'count', 'dockcount', 'elevation','hour_avg', 'month_avg', 'dow_avg','sg_avg']]

In [296]:
df_r.loc[df_r['count'] > 0, 'count'] = 1
df_r = df_r.fillna(0)

In [297]:
y = df_r['count']
X = df_r.drop('count', axis = 1)

In [298]:
weights = float(len(y))/(2*np.bincount(y.astype(int)))
weights = dict(zip([0,1],weights))
rf = RandomForestClassifier(n_jobs=-1, class_weight= weights,  n_estimators= 50, oob_score=True, criterion='entropy')


In [299]:
X_t, X_h,y_t, y_h = train_test_split(X,y, test_size=0.2)

In [300]:
cv = cross_val_score(rf, X_t,y_t, n_jobs=-1,cv = 3, scoring = 'roc_auc')

In [301]:
cv.mean()

0.87145635880509875

In [ ]:
# specify parameters and distributions to sample from
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 9],
              "min_samples_split": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "criterion": ["gini", "entropy"]}
# run randomized search
random_search = GridSearchCV(rf, param_grid=param_grid, n_jobs=-1, verbose = True)

In [ ]:
random_search.fit(X_t,y_t)

In [ ]:
rf = RandomForestClassifier(bootstrap=True,
            class_weight={0: 0.59739085366685307, 1: 3.0669761644679721},
            criterion='entropy', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [302]:
rf.fit(X_t,y_t)

RandomForestClassifier(bootstrap=True,
            class_weight={0: 0.58402051843480718, 1: 3.4754636683654705},
            criterion='entropy', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [303]:
preds = rf.predict(X_h)

In [310]:
preds = pd.Series(preds)

In [305]:
preds = rf.predict_proba(X_h)[:,1]
fpr, tpr, _ = roc_curve(y_h,preds)
plt.plot(fpr,tpr)
plt.show()

In [306]:
preds[preds > .35] = 1
preds[preds <= .35] = 0
confusion_matrix(y_h,preds)

array([[21014,  1680],
       [ 1683,  2083]])

In [311]:
#### examine where model fails

index = df_a.ix[X_h.index, ['terminal','smallgroups','big_cluster','hour','dow','month','hood']]

In [312]:
df_p = pd.concat([X_h.reset_index(),preds,y_h.reset_index(), index.reset_index()], axis = 1)


In [313]:
df_p['correct'] = 0
df_p.ix[df_p['count'] == df_p[0], 'correct'] = 1
df_s = df_p.ix[df_p['count'] == 1, :] 

In [314]:
df_s.groupby(['terminal'])['correct'].mean()


terminal
BT-01     0.618644
BT-03     0.707483
BT-04     0.322581
BT-05     0.473118
CBD-03    0.307692
CBD-04    0.000000
CBD-05    0.663717
CBD-06    0.138462
CBD-07    0.122449
CBD-13    0.481132
CD-01     0.250000
CH-01     0.568627
CH-02     0.659218
CH-03     0.735714
CH-05     0.708075
CH-06     0.606383
CH-07     0.831858
CH-08     0.764331
CH-09     0.151515
CH-12     0.516129
CH-15     0.694030
DPD-01    0.269231
DPD-03    0.000000
EL-01     0.000000
EL-03     0.544118
EL-05     0.119048
FH-01     0.294872
FH-04     0.800000
ID-04     0.045455
PS-04     0.085106
PS-05     0.577465
SLU-01    0.445783
SLU-02    0.710526
SLU-04    0.052632
SLU-07    0.204082
SLU-15    0.711268
SLU-16    0.682927
SLU-17    0.117647
SLU-18    0.520548
SLU-19    0.652174
SLU-20    0.150000
UD-01     0.200000
UD-02     0.000000
UD-04     0.631579
UD-07     0.219512
WF-01     0.507246
WF-04     0.066667
Name: correct, dtype: float64

In [315]:
for i,j in zip(X.columns, rf.feature_importances_):
    print i, j

TEMP 0.130758071482
CLG 0.085450207658
PCP01 0.0155823355456
SPD 0.0948448739228
dockcount 0.0157434811651
elevation 0.0370093660695
hour_avg 0.225918482019
month_avg 0.141628198645
dow_avg 0.151840456662
sg_avg 0.101224526831


In [45]:
df_t = pd.read_csv('data/cleaned/cleaned_trips.csv', parse_dates=['datetime'], infer_datetime_format=True)

In [4]:
df_w = pd.read_csv('data/cleaned/cleaned_weather.csv', parse_dates=['datetime'], infer_datetime_format=True)

In [32]:
datetimes = df_w['datetime']
terminals = df_t['from_station_id'].unique()

In [33]:
import time
t0 = time.clock()
full_index = pd.MultiIndex.from_product([datetimes, terminals], names=['datetime', 'terminal'])
print time.clock() - t0, "seconds process time"

0.007196 seconds process time


In [132]:
df_t['dow'] = map(lambda x: x.dayofweek, df_t['datetime'])
df_t['month'] = map(lambda x: x.month, df_t['datetime'])


In [62]:
df_t['count'] = 1
#df_t= df_t.rename(columns={'from_station_id': 'terminal'})

In [85]:
df_test = df_t.groupby(['terminal','dow','hour','month'])['count'].sum().reset_index()


In [ ]:
df_test.set_index(['datetime', 'terminal'], inplace=True)


In [105]:
df_test = df_t.groupby(['terminal','datetime'])['count'].sum().reset_index()


In [47]:
df_md= df_t[['dockcount_from', 'from_station_id','elevation_from', 'cluster_from', 'ecosystem_from']].drop_duplicates()
df_md.rename(columns={'from_station_id': 'terminal','elevation_from': 'elevation', 'cluster_from': 'cluster', 'ecosystem_from': 'ecosystem', 'dockcount_from': 'dockcount'},inplace=True)
df_ms = df_t[['dockcount_to', 'to_station_id','elevation_to', 'cluster_to', 'ecosystem_to']].drop_duplicates()
df_ms.rename(columns={'to_station_id': 'terminal','elevation_to': 'elevation', 'cluster_to': 'cluster', 'ecosystem_to': 'ecosystem', 'dockcount_to': 'dockcount'}, inplace=True)


In [159]:
df = pd.DataFrame(index=full_index).reset_index()
df = df.merge(df_d, on=['datetime','terminal'], how='left')

In [173]:
df['dow'] = map(lambda x: x.dayofweek, df['datetime'])
df['month'] = map(lambda x: x.month, df['datetime'])

hour_average = dff.groupby(['hour','terminal'])['count'].mean().reset_index()
hour_average.columns = ['hour','terminal', 'hour_avg']

month_average = dff.groupby(['month','terminal'])['count'].mean().reset_index()
month_average.columns = ['month','terminal', 'month_avg']

dow_average = dff.groupby(['dow','terminal'])['count'].mean().reset_index()
dow_average.columns = ['dow','terminal', 'dow_avg']

hour_average.merge(month_average, on='terminal').merge(dow_avg, on='terminal')


KeyError: 'terminal'

In [96]:
df_d = df_t.groupby(['from_station_id','datetime'])['count'].sum().reset_index()
df_d.rename(columns={'from_station_id': 'terminal'}, inplace=True)
df.ix[df['count'].isnull(), 'count'] = 0

In [101]:
hours=[0, 6, 10, 15, 19, 23]

In [160]:
df['hour'] = df['datetime'].map(lambda x: x.hour)
df['date'] = df['datetime'].map(lambda x: x.strftime('%d-%m-%Y'))
df['hour'] = pd.cut(df['hour'], hours, include_lowest=True, right=True, labels=['Early', 'Commute_to_work', 'Afternoon', 'Commute_from_work', 'Night'])
df['hour'] = df['hour'].astype('str')
dff = df.groupby(['date', 'terminal', 'hour'])['count'].sum()
dff =dff.reset_index() 

In [138]:
sum(dff['count'] == 0)/float(len(dff['count']))

0.53067903898719604

In [147]:
#dff.merge(df_md, on='terminal')

In [163]:
df = df.merge(df_w, on='datetime')
df['hour'] = df['datetime'].map(lambda x: x.hour)
df['date'] = df['datetime'].map(lambda x: x.strftime('%d-%m-%Y'))
df['hour'] = pd.cut(df['hour'], hours, include_lowest=True, right=True, labels=['Early', 'Commute_to_work', 'Afternoon', 'Commute_from_work', 'Night'])
df['hour'] = df['hour'].astype('str')
dff = df.groupby(['date', 'hour']).mean()
dff =dff.reset_index() 

In [155]:
dff = df.groupby(['date', 'hour']).mean()
dff =dff.reset_index() 

In [162]:
df1 = dff.copy()

In [179]:
df['date'] = pd.to_datetime(df['date'])
df['dow'] = map(lambda x: x.dayofweek, df['date'])
df['month'] = map(lambda x: x.month, df['date'])

hour_average = df.groupby(['hour','terminal'])['count'].mean().reset_index()
hour_average.columns = ['hour','terminal', 'hour_avg']

month_average = df.groupby(['month','terminal'])['count'].mean().reset_index()
month_average.columns = ['month','terminal', 'month_avg']

dow_average = df.groupby(['dow','terminal'])['count'].mean().reset_index()
dow_average.columns = ['dow','terminal', 'dow_avg']

a = hour_average.merge(month_average, on='terminal').merge(dow_average, on='terminal')


In [192]:
a = month_average.pivot(index='month', columns='terminal', values='month_avg')

In [193]:
a.to_dict()

{'BT-01': {1: 1.6973684210526316,
  2: 1.8627450980392157,
  3: 1.8928571428571428,
  4: 2.133047210300429,
  5: 2.0840336134453783,
  6: 2.1490909090909089,
  7: 2.4964285714285714,
  8: 2.1677852348993287,
  9: 2.0701754385964914,
  10: 1.8251748251748252,
  11: 1.78,
  12: 1.8846153846153846},
 'BT-03': {1: 1.5698324022346368,
  2: 1.4545454545454546,
  3: 1.4860335195530727,
  4: 1.6905829596412556,
  5: 1.7130044843049328,
  6: 1.6175115207373272,
  7: 1.6450216450216451,
  8: 1.7322175732217573,
  9: 1.6359223300970873,
  10: 1.4438775510204083,
  11: 1.3920454545454546,
  12: 1.3888888888888888},
 'BT-04': {1: 1.2307692307692308,
  2: 1.3513513513513513,
  3: 1.375,
  4: 1.4102564102564104,
  5: 1.263157894736842,
  6: 1.4444444444444444,
  7: 1.5538461538461539,
  8: 1.3169014084507042,
  9: 1.436241610738255,
  10: 1.2671232876712328,
  11: 1.2830188679245282,
  12: 1.2413793103448276},
 'BT-05': {1: 1.4459459459459461,
  2: 1.5192307692307692,
  3: 1.5394736842105263,
  4: 1.

In [199]:
df_d = pd.read_csv('data/final/demand_all_features.csv', parse_dates=['date'], infer_datetime_format=True)


,date,terminal,hour,count,TEMP,SPD,PCP01,CLG,VSB,DEWP,SLP,STP,dockcount,elevation,cluster,ecosystem
0,2015-01-01,BT-01,Afternoon,6,29.000000,2.800000,0.000000,722.000000,10.000000,22.000000,1028.520000,1011.500000,18,37.351780,5,0
1,2015-01-01,BT-01,Commute_from_work,1,31.500000,0.750000,0.000000,722.000000,10.000000,23.000000,1028.000000,1010.925000,18,37.351780,5,0
2,2015-01-01,BT-01,Commute_to_work,0,30.500000,7.500000,0.000000,722.000000,10.000000,22.500000,1030.600000,1013.575000,18,37.351780,5,0
3,2015-01-01,BT-01,Early,0,33.571429,9.285714,0.000000,722.000000,10.000000,21.285714,1032.942857,1015.850000,18,37.351780,5,0
4,2015-01-01,BT-01,Night,5,40.250000,3.750000,0.000000,722.000000,10.000000,23.000000,1026.125000,1009.225000,18,37.351780,5,0
5,2015-01-02,BT-01,Afternoon,4,41.400000,4.800000,0.000000,61.600000,3.960000,40.100000,1018.860000,1002.100000,18,37.351780,5,0
6,2015-01-02,BT-01,Commute_from_work,0,43.250000,5.875000,0.010000,52.750000,6.000000,40.375000,1018.275000,1001.625000,18,37.351780,5,0
7,2015-01-02,BT-01,Commute_to_work,0,42.250000,3.250000,0.000000,268.000000,7.000000,39.500000,1019.775000,1003.225000,18,37.351780,5,0
8,2015-01-02,BT-01,Early,1,43.142857,4.000000,0.000000,34.857143,6.557143,39.571429,1021.942857,1005.142857,18,37.351780,5,0
9,2015-01-02,BT-01,Night,0,44.500000,5.500000,0.010000,8.750000,5.725000,41.875000,1018.250000,1001.650000,18,37.351780,5,0
